In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time

ilce_listesi = {
    "5785": "Arnavutköy", "434": "Avcılar", "437": "Bağcılar", "438": "Bahçelievler",
    "435": "Bakırköy", "5786": "Başakşehir", "436": "Bayrampaşa", "439": "Beşiktaş",
    "1077": "Beylikdüzü", "440": "Beyoğlu", "441": "Büyükçekmece", "442": "Çatalca",
    "445": "Eminönü", "443": "Esenler", "5787": "Esenyurt", "444": "Eyüp", "446": "Fatih",
    "447": "Gaziosmanpaşa", "448": "Güngören", "449": "Kağıthane", "450": "Küçükçekmece",
    "5677": "MimarSinan/Büyükçekmece", "451": "Sarıyer", "452": "Silivri", "5765": "Sultangazi",
    "453": "Şişli", "454": "Zeytinburnu", "455": "Adalar", "1722": "Ataşehir", "456": "Beykoz",
    "1244": "Çekmeköy", "457": "Kadıköy", "458": "Kartal", "459": "Maltepe", "460": "Pendik",
    "5788": "Sancaktepe", "461": "Sultanbeyli", "462": "Şile", "463": "Tuzla", "464": "Ümraniye",
    "465": "Üsküdar"
}

for ilce_numarasi, ilce_ismi in ilce_listesi.items():
    toplam_ilan_sayisi = 0
    toplam_basvuru_sayisi = 0

    base_url = f"https://www.kariyer.net/is-ilanlari/{ilce_ismi}?ct=34,82&tt={ilce_numarasi}&date=3g"
    response = requests.get(base_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    sayfa_numarasi_element = soup.select_one("button.page-link[aria-checked='true']")
    sayfa_sayisi = 1
    if sayfa_numarasi_element:
        sayfa_sayisi = int(sayfa_numarasi_element.get("aria-setsize", "1"))

    for sayfa_no in range(1, sayfa_sayisi + 1):
        page_url = f"{base_url}&cp={sayfa_no}"
        page_response = requests.get(page_url)
        page_soup = BeautifulSoup(page_response.content, 'html.parser')

        ilan_kartlari = page_soup.select("a.k-ad-card")

        if ilan_kartlari:
            ilan_sayisi = len(ilan_kartlari)
            toplam_ilan_sayisi += ilan_sayisi

            for ilan_karti in ilan_kartlari:
                ilan_detay_url_relative = ilan_karti['href']
                ilan_detay_url_absolute = urljoin("https://www.kariyer.net", ilan_detay_url_relative)

                ilan_detay_response = requests.get(ilan_detay_url_absolute)
                ilan_detay_soup = BeautifulSoup(ilan_detay_response.content, 'html.parser')

                basvuru_sayisi_tagi = ilan_detay_soup.select_one("p.mb-0 span[data-v-5fb4fcba='']")

                if basvuru_sayisi_tagi:
                    basvuru_sayisi_text = basvuru_sayisi_tagi.parent.text.strip()
                    basvuru_sayisi_sadece_rakamlar = ''.join(filter(str.isdigit, basvuru_sayisi_text))

                    try:
                        basvuru_sayisi_int = int(basvuru_sayisi_sadece_rakamlar)
                        toplam_basvuru_sayisi += basvuru_sayisi_int
                    except ValueError:
                        pass

    print(f"İlçe İsmi: {ilce_ismi}")
    print(f"Toplam İlan Sayısı: {toplam_ilan_sayisi}")
    print(f"Toplam Başvuru Sayısı: {toplam_basvuru_sayisi}")
    print()

In [ ]:
import concurrent.futures
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import matplotlib.pyplot as plt
import seaborn as sns

ilce_listesi = {
    "5785": "Arnavutköy", "434": "Avcılar", "437": "Bağcılar", "438": "Bahçelievler",
    "435": "Bakırköy", "5786": "Başakşehir", "436": "Bayrampaşa", "439": "Beşiktaş",
    "1077": "Beylikdüzü", "440": "Beyoğlu", "441": "Büyükçekmece", "442": "Çatalca",
    "445": "Eminönü", "443": "Esenler", "5787": "Esenyurt", "444": "Eyüp", "446": "Fatih",
    "447": "Gaziosmanpaşa", "448": "Güngören", "449": "Kağıthane", "450": "Küçükçekmece",
    "5677": "MimarSinan/Büyükçekmece", "451": "Sarıyer", "452": "Silivri", "5765": "Sultangazi",
    "453": "Şişli", "454": "Zeytinburnu", "455": "Adalar", "1722": "Ataşehir", "456": "Beykoz",
    "1244": "Çekmeköy", "457": "Kadıköy", "458": "Kartal", "459": "Maltepe", "460": "Pendik",
    "5788": "Sancaktepe", "461": "Sultanbeyli", "462": "Şile", "463": "Tuzla", "464": "Ümraniye",
    "465": "Üsküdar"
}

ilceler = []
toplam_ilan_sayilari = []
toplam_basvuru_sayilari = []

def process_ilce(ilce_numarasi, ilce_ismi):
    toplam_ilan_sayisi = 0
    toplam_basvuru_sayisi = 0

    base_url = f"https://www.kariyer.net/is-ilanlari/{ilce_ismi}?ct=34,82&tt={ilce_numarasi}&date=3g"
    response = requests.get(base_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    sayfa_numarasi_element = soup.select_one("button.page-link[aria-checked='true']")
    sayfa_sayisi = 1
    if sayfa_numarasi_element:
        sayfa_sayisi = int(sayfa_numarasi_element.get("aria-setsize", "1"))

    for sayfa_no in range(1, sayfa_sayisi + 1):
        page_url = f"{base_url}&cp={sayfa_no}"
        page_response = requests.get(page_url)
        page_soup = BeautifulSoup(page_response.content, 'html.parser')

        ilan_kartlari = page_soup.select("a.k-ad-card")

        if ilan_kartlari:
            ilan_sayisi = len(ilan_kartlari)
            toplam_ilan_sayisi += ilan_sayisi

            for ilan_karti in ilan_kartlari:
                ilan_detay_url_relative = ilan_karti['href']
                ilan_detay_url_absolute = urljoin("https://www.kariyer.net", ilan_detay_url_relative)

                ilan_detay_response = requests.get(ilan_detay_url_absolute)
                ilan_detay_soup = BeautifulSoup(ilan_detay_response.content, 'html.parser')

                basvuru_sayisi_tagi = ilan_detay_soup.select_one("p.mb-0 span[data-v-5fb4fcba='']")

                if basvuru_sayisi_tagi:
                    basvuru_sayisi_text = basvuru_sayisi_tagi.parent.text.strip()
                    basvuru_sayisi_sadece_rakamlar = ''.join(filter(str.isdigit, basvuru_sayisi_text))

                    try:
                        basvuru_sayisi_int = int(basvuru_sayisi_sadece_rakamlar)
                        toplam_basvuru_sayisi += basvuru_sayisi_int
                    except ValueError:
                        pass

    return ilce_ismi, toplam_ilan_sayisi, toplam_basvuru_sayisi

with concurrent.futures.ThreadPoolExecutor() as executor:
    future_to_ilce = {executor.submit(process_ilce, ilce_numarasi, ilce_ismi): ilce_ismi for ilce_numarasi, ilce_ismi in ilce_listesi.items()}
    for future in concurrent.futures.as_completed(future_to_ilce):
        ilce_ismi, toplam_ilan_sayisi, toplam_basvuru_sayisi = future.result()
        ilceler.append(ilce_ismi)
        toplam_ilan_sayilari.append(toplam_ilan_sayisi)
        toplam_basvuru_sayilari.append(toplam_basvuru_sayisi)

        # Görselleştirme
        plt.figure(figsize=(15, 8))
        sns.barplot(x=ilceler, y=toplam_ilan_sayilari)
        plt.xticks(rotation=90)
        plt.title('İlçe Bazında Toplam İlan Sayısı')
        plt.show()

        plt.figure(figsize=(15, 8))
        sns.barplot(x=ilceler, y=toplam_basvuru_sayilari)
        plt.xticks(rotation=90)
        plt.title('İlçe Bazında Toplam Başvuru Sayısı')
        plt.show()